In [115]:
import pandas as pd
import datetime
from configparser import ConfigParser
import json, requests
from pysimplicate import Simplicate
from hplib.dbclass import dbClass

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )
pd.set_option('display.max_columns', 300)

In [68]:
# Nieuwe methode. 
# 1. Alle active projecten
# 2. De diensten daarvan
# 3. Omzet -/- correcties berekenen
# 4. Ophalen wat er is gefactureerd

In [123]:
# 1. Alle active projecten
# 2. De diensten daarvan
def simplicate_projects_and_services():
    # Get the list of services 
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    services_json = sim.service({'track_hours':True})
    services = sim.to_pandas(services_json) \
     .query(f'invoice_method != "Subscription" & status != "flushed" & (invoice_method != "FixedFee" | status!="invoiced")')\
     [['project_id','status','id','name','start_date','end_date']]\
     .rename(columns={'id':'service_id', 'name':'service', 'status':'service_status'})
    
    # Same with the list of projects
    projects_json = sim.project({'status':'tab_pactive'})
    projects = sim.to_pandas(projects_json)[['id','project_number','name','organization_name','project_manager_name']]\
        .rename(columns={'id':'project_id', 'name':'project_name'})

    # Join them
    project_service = pd.merge(services,projects,on=['project_id'])
    return project_service

projects_and_services = simplicate_projects_and_services()
projects_and_services[:3]

,project_id,service_status,service_id,service,start_date,end_date,project_number,project_name,organization_name,project_manager_name
0,project:13174969648b84f04c13c77ab857ae53,open,service:6680f6a811f75b53d8107a68a60f150e,Strippenkaart 1 (gratis bij oplevering project),NaN,2020-04-03,MLA-2,MLA Strippenkaart Montessori Lyceum,Montessori Lyceum Amsterdam,Gerben van Dijk
1,project:07dd1b170f0e1fb4feaad60b7a7437df,open,service:a2c8abc0cfaa4746d8107a68a60f150e,Budgetoverschot 2019 van 10 uur,2020-01-01,NaN,ROCK-4,Rockstart Agile 2020,Rockstart Enterprises B.V.,Stefan Roovers
2,project:8166d3b4cc16aa654c13c77ab857ae53,open,service:5b807d06e583857ed8107a68a60f150e,Strippenkaart 9,2020-10-01,2021-10-29,NLA-1,NLA Strippenkaart,Next Level Academy B.V.,Gerben van Dijk


In [125]:
active_services = projects_and_services['service_id'].tolist()
active_services_string = '("' + '","'.join( active_services) + '")'
print( len(active_services), 'active services')

197 active services


In [126]:
# 3. Omzet berekenen
# Database bij AWS
db = dbClass.from_inifile('../credentials.ini', section='aws-dashboard')
untilday = '2021-12-01'
query = f'''select organization, project_name, project_number, project_id, service_name, service_id, 
           sum(turnover) as turnover from timesheet
           where organization not in ('Oberon','Travelbase') and service_id in {active_services_string}
             and day<"{untilday}"
           group by service_id
           order by organization, service_id'''
services_with_turnover = pd.read_sql_query(query, db.db)
services_with_turnover[:3]

,organization,project_name,project_number,project_id,service_name,service_id,turnover
0,Alex de Rollende Fietsenmaker,Alex en Iris 2021,ALEX-2,project:3547a32f391eae72feaad60b7a7437df,Strippenkaart Alex en Iris 2021,service:c0c8ba1993f7e08612278dc9a55a75e0,633.75
1,Amsterdam Internet Exchange BV.,AMS-IX SLA,AMS-1,project:b3f4f7e167cff85cfeaad60b7a7437df,Service Basic,service:6e599f61f95cd84ddbbc6eee40e2b5f7,25370.00
2,AndersWinst Italia srl,Kpito - SLA,KPITO-1,project:3d5fc28224ef4516feaad60b7a7437df,Service Basic,service:47836380d173f299d8107a68a60f150e,330.00


In [127]:
# 4. Ophalen wat er is gefactureerd
invoiced = sim.invoiced_per_service({"from_date": "2021-01-01", "until_date": untilday})

In [128]:
services_with_turnover['invoiced'] = services_with_turnover.apply(lambda row: invoiced[row['service_id']], axis=1)
services_with_turnover['ohw'] = services_with_turnover['turnover'] - services_with_turnover['invoiced']
services_with_turnover.sort_values(by='ohw', ascending=False)

,organization,project_name,project_number,project_id,service_name,service_id,turnover,invoiced,ohw
24,Collabhouse BV,Collabhouse platform,COL-1,project:85b01899fbac9f06feaad60b7a7437df,Fase 4 - Development sprint 7,service:7a4d08a692b34d648ae8092ac3a775c4,27685.65,0.000,27685.650
37,Easybroker B.V.,Preparation sprints,EASY-4,project:001d444b2544da20feaad60b7a7437df,Preparation sprints,service:2d56a05cf1d275daf0d06734914aca76,23948.76,0.000,23948.760
118,Vision Health Care,Platform sprints,VHC-1,project:be89fc3d5a587007feaad60b7a7437df,Sprint 3,service:d04af9faa1fd3bdd827610813b27dd96,18283.14,0.000,18283.140
71,Sprout Money BV,Gini Capital,GINI,project:a8485df118750546feaad60b7a7437df,Development of Gini Capital,service:10fe9c300a75ae48783d0f4c8b4f4078,17597.50,0.000,17597.500
38,Easybroker B.V.,Design fases,EASY-2,project:b4d3ad8257533e1bfeaad60b7a7437df,UX/UI design,service:35c0916779b29a9db673c98d2eab4ca5,15066.25,0.000,15066.250
...,...,...,...,...,...,...,...,...,...
63,Oncode,Oncode Community aanpassingen,ONC-3,project:d173d76afe4f8dd9feaad60b7a7437df,Development Sprint,service:0fbbfc8fb1f7ab9e8ae8092ac3a775c4,9968.75,9968.750,0.000
62,Oerol,Platform,OER-1,project:216e75fe9b88e377feaad60b7a7437df,Development Sprint Oerol XTR,service:e803c88607102941b61e8ec4cf9407af,2520.00,2520.000,0.000
61,Oerol,Platform,OER-1,project:216e75fe9b88e377feaad60b7a7437df,Development Sprint Fiona koppeling,service:b104600ec6756244f0d06734914aca76,20330.00,20330.000,0.000
79,Sprout Money BV,Value Jagers Agile,VJ2021,project:8a11f2a1ccca7569feaad60b7a7437df,Value Jagers - April 2021,service:809448b0d2c79f8812278dc9a55a75e0,3761.25,3761.250,0.000


In [129]:
services_with_turnover['ohw'].sum()

158921.4150000001

In [130]:
services_with_turnover

,organization,project_name,project_number,project_id,service_name,service_id,turnover,invoiced,ohw
0,Alex de Rollende Fietsenmaker,Alex en Iris 2021,ALEX-2,project:3547a32f391eae72feaad60b7a7437df,Strippenkaart Alex en Iris 2021,service:c0c8ba1993f7e08612278dc9a55a75e0,633.75,0.0000,633.7500
1,Amsterdam Internet Exchange BV.,AMS-IX SLA,AMS-1,project:b3f4f7e167cff85cfeaad60b7a7437df,Service Basic,service:6e599f61f95cd84ddbbc6eee40e2b5f7,25370.00,25370.0000,0.0000
2,AndersWinst Italia srl,Kpito - SLA,KPITO-1,project:3d5fc28224ef4516feaad60b7a7437df,Service Basic,service:47836380d173f299d8107a68a60f150e,330.00,330.0000,0.0000
3,BAM,Homestudios Traject 2021,BAM-1,project:b702bba66765ebadfeaad60b7a7437df,Development Sprints Maart,service:1dee6ea1f6f1c57c827610813b27dd96,30013.03,30013.0125,0.0175
4,BAM,Homestudios Traject 2021,BAM-1,project:b702bba66765ebadfeaad60b7a7437df,Development Sprints Januari,service:48b5e7e7e3fc61d7d8107a68a60f150e,23247.27,23247.2625,0.0075
...,...,...,...,...,...,...,...,...,...
131,VVV Texel,Website,TEX-1,project:2e7d1588e87a48a8feaad60b7a7437df,Development Sprint juli,service:ee575fdd879b74c08ae8092ac3a775c4,2145.02,2145.0000,0.0200
132,VVV Waterland van Friesland,Travelbase Doorontwikkeling Q3 en Q4 2021,VWF-3,project:87872cbd3874d935feaad60b7a7437df,Development Sprint Doorontwikkeling,service:ed6134dfc6bdf519dbbc6eee40e2b5f7,231.26,0.0000,231.2600
133,Warner Music Benelux B.V.,Maestro Music - SLA,MM-1,project:6c438f3a9abac41bfeaad60b7a7437df,Service Basic,service:665ae06794a715c4f0d06734914aca76,27.50,0.0000,27.5000
134,Warner Music Benelux B.V.,Maestro Maintenance & Support,WARN-1,project:8c9492f7a76737be4c13c77ab857ae53,Maintenance and Support o.b.v. nacalculatie,service:c863795a0589a3428ae8092ac3a775c4,1072.50,1072.5000,0.0000


In [131]:
per_project = services_with_turnover.groupby(['project_number', 'project_name'])[['turnover','invoiced','ohw']].sum()
per_project.sort_values(by='ohw', ascending=False).query('ohw>1000 | ohw<-1000')

,,turnover,invoiced,ohw
project_number,project_name,,,
COL-1,Collabhouse platform,221530.75,193845.000,27685.750
VHC-1,Platform sprints,42545.69,15709.375,26836.315
EASY-4,Preparation sprints,23948.76,0.000,23948.760
GINI,Gini Capital,17597.50,0.000,17597.500
EASY-2,Design fases,15066.25,0.000,15066.250
TUI-3,TLP nov/dec,14600.00,0.000,14600.000
TER-2,Website VVV Terschelling,11116.89,0.000,11116.890
LAND-2,Land Life,12746.25,6902.500,5843.750
THIE-25,Examenbundel Continuous Improvements (CI),15220.00,11710.000,3510.000


In [132]:
per_project.sort_values(by='ohw', ascending=False)

,,turnover,invoiced,ohw
project_number,project_name,,,
COL-1,Collabhouse platform,221530.75,193845.000,27685.750
VHC-1,Platform sprints,42545.69,15709.375,26836.315
EASY-4,Preparation sprints,23948.76,0.000,23948.760
GINI,Gini Capital,17597.50,0.000,17597.500
EASY-2,Design fases,15066.25,0.000,15066.250
...,...,...,...,...
AME-1,VVV Ameland - SLA,137.50,137.500,0.000
ESCH-1,Van Esch - SLA,7020.00,7020.000,0.000
KPITO-1,Kpito - SLA,330.00,330.000,0.000


# Oude code

In [4]:
# Simplicate projectstatus call. Obsolete

def project_status_data(date_str=None):
    session = requests.Session()
    login_url = 'https://oberon.simplicate.com/site/login'
    login_data = {
        'LoginForm[username]': ini['simplicate']['username'],
        'LoginForm[password]': ini['simplicate']['password'],
    }
    report_url = (
        'https://oberon.simplicate.com/v1/reporting/process/reloadData?q={"page":"process","project_status":["active"],"myorganizations":["2"]}'
    ) # myorganizations 1: Qikker, 2: Oberon, 3: Travelbase
    if date_str:
        report_url = report_url[:-1] + f',"date":"{date_str}"' + '}'

    session.post(login_url, login_data)
    
    try:
        json_data = session.get(report_url).json()
    except ConnectionResetError:
        log.log_error('simplicate.py', 'onderhanden_werk', 'Connection reset by Simplicate')
        return 0
    except json.decoder.JSONDecodeError:
        log.log_error('simplicate.py', 'onderhanden_werk', 'JSON DecodeError')
        return 0
    return json_data
#project_status_json = project_status_data()

In [61]:
# Obsolete
def get_project_status_dataframe():
    
    def ohw_type( s ):
        if s['payment_type'] == 'Vaste prijs':
            if s['Budget']:
                return 'Strippenkaart'
            else:
                return 'Fixed'
        else:
            return 'Normal'
        return int(result)

    def calculate_ohw( s ):
        # Calculate OHW based on ohw_type
        if s['ohw_type'] == 'Strippenkaart':
            result = -s['Restant budget']
        elif s['ohw_type'] == 'Fixed':
            result = s['Verwacht'] - s['Gefactureerd']
        else:
            result = s['Besteed'] + s['Correcties'] + s[' Besteed'] - s['Gefactureerd'] + s['Marge gerealiseerd']
        return int(result)

    def parse_project_status_json(project_status_json):
        headers = [col['title'] for col in project_status_json['table']['columns']]
        res = []
        for row in project_status_json['table']['rows'][1:]: # De eerste rij is leeg    
            row_values = [rc[0]['value'] for rc in row['columns']]
            subrows = row['subrows']
            for subrow in subrows:
                subrow_values = [src[0]['value'] for src in subrow['columns']]
                rec= {'project':row['headers'][0]['value'], 'service':subrow['headers'][0]['value']}
                for key,val in zip(headers,subrow_values):
                    rec[key] = val
                res += [rec]
        project_status_dataframe = pd.DataFrame( res ).replace('-',0)
        return project_status_dataframe

    def enhance_project_status_dataframe( prs: pd.DataFrame ):
        prs['project_number'] = prs.apply(lambda s: s['project'].rsplit('(')[-1].split(')')[0], axis=1 ) # Between () in project
        prs['payment_type'] = prs.apply(lambda s: s['service'].rsplit('[')[-1].split(']')[0].strip(), axis=1 ) # Between [] in service
        prs['service'] = prs.apply(lambda s: s['service'].split(' [')[0].strip(), axis=1 ) # Chop off [Vaste prijs]
        prs['ohw_type'] = prs.apply( ohw_type, axis=1) # Strippenkaart, Fixed or Normal
        prs['OHW2'] = prs.apply( calculate_ohw, axis=1) # Calculate based on ohw_type
        return prs
    

    json = project_status_json # !!project_status_data()
    df = parse_project_status_json(json)
    enhance_project_status_dataframe( df )
    return df

#prs = get_project_status_dataframe()
#prs[:3]

,project,service,Huidige status,Begindatum,Einddatum,Budget,Besteed,Gepland,Correcties,Restant budget,Budget,Marge,Verkoopbudget,Besteed,Afgeboekt,Marge gerealiseerd,Restant budget,Verkoop gerealiseerd,Bruto marge,Kostprijs mdw.,Projectresult.,Verwacht,Gefactureerd,OHW,project_number,payment_type,ohw_type,OHW2
0,Alda Nederland B.V. - ALDA General Strippenkaa...,Resterende budget van oude strippenkaart,0,0,0,1148,1147.5,0,0.0,0.5,0,0,0,0,0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,ALDA-13,Vaste prijs,Strippenkaart,0
1,Alda Nederland B.V. - ALDA General Strippenkaa...,Strippenkaart 1 2019,0,0,0,4000,4050.0,0,-50.0,0.0,0,0,0,0,0,0.0,0,0.0,4000.0,0,4000.0,4000.0,4000.0,0.0,ALDA-13,Vaste prijs,Strippenkaart,0
2,Alda Nederland B.V. - ALDA General Strippenkaa...,Strippenkaart 1 2020,0,0,0,4000,3975.0,0,0.0,25.0,0,0,0,0,0,0.0,0,0.0,4000.0,0,4000.0,4000.0,4000.0,-25.0,ALDA-13,Vaste prijs,Strippenkaart,-25


In [63]:
def simplicate_projects_and_services():
    # Get the list of services 
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    # !! services_json = sim.service()
    services = sim.to_pandas(services_json) \
     .query(f'invoice_method != "Subscription" & track_hours == True & status != "flushed"')\
     [['project_id','status','id','name','track_hours','start_date','end_date']]\
     .rename(columns={'id':'service_id', 'name':'service', 'status':'service_status'})

    # Same with the list of projects
    # !! projects_json = sim.project({'status':'tab_pactive'})
    projects = sim.to_pandas(projects_json)[['id','project_number','name','organization_name','project_manager_name']]\
        .rename(columns={'id':'project_id', 'name':'project_name'})

    # Join them
    project_service = pd.merge(services,projects,on=['project_id'])
    return project_service

simplicate_projects_and_services()[:3]

,project_id,service_status,service_id,service,track_hours,start_date,end_date,project_number,project_name,organization_name,project_manager_name
0,project:8166d3b4cc16aa654c13c77ab857ae53,invoiced,service:5c5154c7a6aa6149d8107a68a60f150e,Strippenkaart 3,True,2018-10-01,2018-10-03,NLA-1,NLA Strippenkaart,Next Level Academy B.V.,Gerben van Dijk
1,project:8166d3b4cc16aa654c13c77ab857ae53,invoiced,service:0bfa23c9ec9ad558d8107a68a60f150e,Strippenkaart 4,True,2018-11-05,2018-11-26,NLA-1,NLA Strippenkaart,Next Level Academy B.V.,Gerben van Dijk
2,project:8166d3b4cc16aa654c13c77ab857ae53,invoiced,service:edf5ac642517e999d8107a68a60f150e,Strippenkaart 5,True,NaN,2019-02-11,NLA-1,NLA Strippenkaart,Next Level Academy B.V.,Gerben van Dijk


In [77]:
def ohw_list():
    rename_columns = {'project_number_y':'project_number', 'Marge gerealiseerd':'verkoopmarge', 'OHW2':'ohw', 'project_manager_name':'pm'}
    return_columns = ['project_number', 'service', 'Besteed', 'Correcties', 'Gefactureerd', 'verkoopmarge', 'ohw', 'ohw_type', 'organization_name', 'project_name', 'project_id', 'service_id','pm','start_date','end_date']

    project_status_dataframe = get_project_status_dataframe()
    projects_and_services = simplicate_projects_and_services()
    merged = pd.merge(project_status_dataframe,projects_and_services, on=['project_number', 'service']) \
           .rename(columns=rename_columns) \
           [return_columns]
    merged.sort_values(by='project_number')
    return merged
    
ohw_list()[:3]

,project_number,service,Besteed,Correcties,Gefactureerd,verkoopmarge,ohw,ohw_type,organization_name,project_name,project_id,service_id,pm,start_date,end_date
0,ALDA-13,Strippenkaart 1 2019,4050.00,-50.0,4000.0,0.0,0,Strippenkaart,Alda Nederland B.V.,ALDA General Strippenkaart,project:20915e733ad449a6feaad60b7a7437df,service:71150bc5a913075fd8107a68a60f150e,Gerben van Dijk,2019-03-28,2019-03-07
1,ALDA-13,Strippenkaart 1 2020,3975.00,0.0,4000.0,0.0,-25,Strippenkaart,Alda Nederland B.V.,ALDA General Strippenkaart,project:20915e733ad449a6feaad60b7a7437df,service:55647084f3050b86d8107a68a60f150e,Gerben van Dijk,NaN,2020-03-10
2,ALEX-2,Strippenkaart Alex en Iris 2021,633.75,0.0,0.0,0.0,633,Normal,Alex de Rollende Fietsenmaker,Alex en Iris 2021,project:3547a32f391eae72feaad60b7a7437df,service:c0c8ba1993f7e08612278dc9a55a75e0,Gerben van Dijk,2021-02-10,NaN


In [78]:
ohw_list()['ohw'].sum()

110862

In [80]:
# Get project numbers of all projects with > +/-1000 OWH
ohw_projects = set(ohw_list().groupby(['project_number']).sum('ohw').query('abs(ohw) >= 1000').reset_index()['project_number'])
ohw_list().query( 'abs(ohw) > 0 & project_number in @ohw_projects').sort_values(by='project_number')

,project_number,service,Besteed,Correcties,Gefactureerd,verkoopmarge,ohw,ohw_type,organization_name,project_name,project_id,service_id,pm,start_date,end_date
3,AMS-1,Service Basic,39737.500000,-15180.00,22852.500,0.0,1705,Normal,Amsterdam Internet Exchange BV.,AMS-IX SLA,project:b3f4f7e167cff85cfeaad60b7a7437df,service:6e599f61f95cd84ddbbc6eee40e2b5f7,Caspar Geerlings,2021-01-01,NaN
142,BEN-1,Development Sprints,138876.875000,1747.50,138351.875,0.0,2272,Normal,T-Mobile Netherlands B.V.,BEN App,project:921a9486c1cbd8e6feaad60b7a7437df,service:4cb3b46fbdec24cad8107a68a60f150e,Gert Braun,2021-01-01,NaN
101,BITF-1,Design: visual style & branding,2592.500000,0.00,3740.000,0.0,-1147,Strippenkaart,Sprout Money BV,BitFirst Exchange,project:6c7adeccdadb36f4feaad60b7a7437df,service:42401517deb07c15827610813b27dd96,Stefan Roovers,2021-01-18,2021-03-08
103,BITF-1,Setup / support Binance portal (strippenkaart 2),2337.500000,0.00,3400.000,0.0,-1062,Strippenkaart,Sprout Money BV,BitFirst Exchange,project:6c7adeccdadb36f4feaad60b7a7437df,service:b104600ec6756244dbbc6eee40e2b5f7,Stefan Roovers,2021-02-17,2021-03-03
100,BITF-1,Bitfirst support (strippenkaart 4),3081.250000,0.00,3400.000,0.0,-318,Strippenkaart,Sprout Money BV,BitFirst Exchange,project:6c7adeccdadb36f4feaad60b7a7437df,service:f846d5882587027d783d0f4c8b4f4078,Stefan Roovers,2021-06-01,2021-07-13
99,BITF-1,Bitfirst support (strippenkaart 3),3272.500000,0.00,3400.000,0.0,-127,Strippenkaart,Sprout Money BV,BitFirst Exchange,project:6c7adeccdadb36f4feaad60b7a7437df,service:e803c88607102941b673c98d2eab4ca5,Stefan Roovers,2021-05-21,2021-05-21
19,BRA-2,Service Team,1395.000000,0.00,0.000,0.0,1395,Normal,De Brauw Blackstone Westbroek,SLA,project:1b594af55d20b05bfeaad60b7a7437df,service:7eea09b579ba1869827610813b27dd96,Caspar Geerlings,2021-07-01,NaN
7,CAP-12,Research,0.000000,0.00,0.000,0.0,1200,Fixed,Capital A,Progress IT due diligence (project Martini),project:710f57c6bb18753dfeaad60b7a7437df,service:2d56a05cf1d275dab48628b99c52fe43,Hans-Peter Harmsen,2021-09-01,2021-12-01
143,CEO-1,Service Plus,11340.000000,-1125.00,8392.500,0.0,1822,Normal,T-Mobile Netherlands B.V.,CEO App - SLA,project:6e0295b631d25503feaad60b7a7437df,service:efb97a904673e2eeb48628b99c52fe43,Caspar Geerlings,2021-01-01,NaN
17,COL-1,Fase 4 - Development sprint 5,6633.125000,0.00,0.000,0.0,6633,Normal,Collabhouse BV,Collabhouse platform,project:85b01899fbac9f06feaad60b7a7437df,service:35c0916779b29a9ddbbc6eee40e2b5f7,Huong Nguyen,2021-10-25,2021-11-05


In [44]:

# TODO: Specifieke datum

len(merged)

197